<a href="https://colab.research.google.com/github/AlfiRam/Computer-Vision-Fall-Detection-Assignment/blob/main/Fall_Detection_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

POSE_CONFIDENCE_THRESHOLD = 0.5
with mp_pose.Pose(
    min_detection_confidence=POSE_CONFIDENCE_THRESHOLD,
    min_tracking_confidence=POSE_CONFIDENCE_THRESHOLD
) as pose:
    video_path = '/content/drive/MyDrive/UNI/SEMESTER 4/comvis/Alfi Falling Video Landscape 720.mp4'

    cap = cv2.VideoCapture(video_path)

    prev_centroid_y = None
    fall_frames = 0
    fall_threshold = 10
    centroid_drop_threshold = 0.2
    frame_counter = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_counter += 1
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(image)

        #Check if any pose is detected
        if results.pose_landmarks:
            #Get the bounding box coordinates of the person
            bbox_min_x = min(landmark.x for landmark in results.pose_landmarks.landmark)
            bbox_max_x = max(landmark.x for landmark in results.pose_landmarks.landmark)
            bbox_min_y = min(landmark.y for landmark in results.pose_landmarks.landmark)
            bbox_max_y = max(landmark.y for landmark in results.pose_landmarks.landmark)
            bbox_width = (bbox_max_x - bbox_min_x) * frame.shape[1]
            bbox_height = (bbox_max_y - bbox_min_y) *frame.shape[0]

            #Calculate the centroid of the person
            centroid_x = bbox_min_x + (bbox_width / 2)
            centroid_y = bbox_min_y + (bbox_height / 2)

            centroid_drop = 0

            #Check if centroid position moved down
            if prev_centroid_y is not None:
                centroid_drop = prev_centroid_y - centroid_y
                significant_drop = centroid_drop > centroid_drop_threshold
                fall_frames += 1 if significant_drop else 0
            else:
                fall_frames = 0

            #Check if person has fallen based on the condition:
            is_falling = fall_frames >= fall_threshold or bbox_width > bbox_height

            #Display  bounding box
            cv2.rectangle(frame, (int(bbox_min_x * frame.shape[1]), int(bbox_min_y * frame.shape[0])),
                          (int(bbox_max_x * frame.shape[1]), int(bbox_max_y * frame.shape[0])), (0, 255, 0), 2)

            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

            cv2_imshow(frame)

            print("frame number: ",frame_counter, " centroid drop: ", centroid_drop, " fall frames: ", fall_frames, " fall threshold: ", fall_threshold, " bounding box width: ", bbox_width, " bounding box height: ", bbox_height)
            if is_falling:
                print("\033[1m\033[92mFALL DETECTED!\033[0m")
            else:
                print("\033[1m\033[91mno fall detected\033[0m")

        prev_centroid_y = centroid_y

    # Release the video capture and close the windows
    cap.release()
    cv2.destroyAllWindows()
